In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
 
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [2]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


## Wczytywanie danych

In [14]:
df = pd.read_hdf("data/car.h5")
df.shape

(106494, 155)

In [17]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

#Dummy Model

In [18]:
df.select_dtypes(np.number).columns #sprawdzanie jakie kolumny są numerami, czyli (float/int)

Index(['price_value', 'car_id'], dtype='object')

In [26]:
feats = ['car_id']
X = df[ feats ].values # [[]] <- lista cech
y = df['price_value'].values # [] <- wektor

model = DummyRegressor() # tworzenie modelu
model.fit(X, y)          # trening modelu
y_pred = model.predict(X)# prognozowanie 

mae(y, y_pred)
# oblicza wartosc srednia delt predykcji od wartosci prawdziwej

39465.934630440985

In [28]:
[x for x in df.columns if 'price' in x]
# sprawdzenie kolumn zawierające słowa 'price'

['price_currency', 'price_details', 'price_value']

In [32]:
df['price_currency'].value_counts()
#z reguły powinno się wartości sprowadzać do jednej jednostki

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [35]:
df = df[df['price_currency'] != 'EUR' ]
df.shape

(106290, 155)

#Features

In [39]:
df['param_color'].factorize()[0]
#factorize pozwala na przypisanie id dla poszczegolnych wartosic
#celem jest zamiana str "czarny" -> "12", bo model pracuje na wartosciach

(array([-1, -1, -1, ..., -1, -1, -1]),
 Index(['Czarny', 'Srebrny', 'Złoty', 'Bordowy', 'Biały', 'Szary', 'Beżowy',
        'Zielony', 'Fioletowy', 'Niebieski', 'Inny kolor', 'Brązowy', 'Żółty'],
       dtype='object'))

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list):continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [44]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [50]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
# wieksza glebokosc modelu moze sprawic, ze model bedzie przeuczony
# czyli bedzie bardzo dobrze analizowac dane ktore zostaly mu dostarczone do nauki
# ale moze sobie nie poradzic z analizowaniem nowych danych
# wynika to z tego, ze w danych do nauki moga znajdowac sie bledy
# i algorytm na podstawie tych bledow znajdzie zaleznosc, ktora jest falszywa
scores = cross_val_score(model, X, y ,cv = 3, scoring="neg_mean_absolute_error")

np.mean(scores)


-19650.45289201444

In [53]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X,y)

imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names = cat_feats)

Weight,Feature
0.1956 ± 0.0054,param_faktura-vat__cat
0.1903 ± 0.0027,param_napęd__cat
0.1815 ± 0.0087,param_stan__cat
0.1512 ± 0.0078,param_rok-produkcji__cat
0.0928 ± 0.0019,param_skrzynia-biegów__cat
0.0607 ± 0.0051,param_moc__cat
0.0436 ± 0.0015,feature_kamera-cofania__cat
0.0191 ± 0.0022,param_pojemność-skokowa__cat
0.0162 ± 0.0004,feature_bluetooth__cat
0.0110 ± 0.0007,feature_łopatki-zmiany-biegów__cat


In [0]:
!git add day3_simple_model